In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pwd

Mounted at /content/drive
/content


In [2]:
%cd /content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/

/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master


In [3]:
!pip install -r "/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.4/687.4 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements

In [4]:
!pip install ruamel.yaml
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.3 MB/s eta 0:00:00


In [5]:
import os
import math
from tqdm import tqdm
import time
import pyarrow as pa
import pyarrow.parquet as pq


import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import plotly.express as px

if os.getcwd().split(os.sep)[-1] == "notebooks":
    os.chdir("../")

# import helper functions coming with this project
import helper.io as io_help
import helper.tracking_data as td_help
import helper.Visualizer as viz_help
import helper.game_models as gm_help

# this is very useful as it makes sure that always all columns and rows of a data frame are displayed
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# **실험실 ☔**

: game 1에 대해서 우선적으로 실행. 추후 game 2, 3에 대하여 진행 예정

# **0) 데이터 전처리**

**0-1) Tracking Data 전처리**

In [6]:
df_match_1 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match1.csv")
df_match_2 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match2.csv")
df_match_3 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match3.csv")

In [7]:
df_match_1.head()

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,ball_x,ball_y
0,1,1,0.1,1,1,B,NaN,B19,PASS,0.100440,34.731360,0.333290,-0.035174,0.335140,-0.625875,35.259840,47.031840,-0.251670,-2.273060,2.286950,-0.059494,36.397080,35.181360,-0.245772,0.820850,0.856854,-2.339236,33.401160,25.580880,0.086596,0.185292,0.204529,3.577530,34.707960,15.308640,0.906415,-2.713272,2.860670,0.463751,44.381520,52.264080,0.211816,-2.904561,2.912274,-0.290737,45.033840,34.446960,0.105120,0.959609,0.965349,-0.175408,42.255000,23.436000,1.116675,-3.071419,3.268115,0.208952,49.019040,15.245280,0.056168,-3.032358,3.032878,1.306999,56.912760,27.345600,0.183449,-5.193607,5.196846,0.145989,59.662440,31.153680,1.015902,-4.618719,4.729125,-2.593574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.731495,34.172730,-1.339970,-0.019173,1.340108,-0.864164,63.064440,14.971680,-2.051766,0.765264,2.189834,-2.652163,73.070640,33.631200,-1.938117,-1.679287,2.564430,-1.044052,72.694800,55.062720,-2.074982,-1.893343,2.808968,0.341137,44.045640,44.298000,-1.353995,-3.043234,3.330852,0.230280,49.109760,27.870480,-1.115580,-3.675937,3.841488,0.074951,60.436800,48.798000,1.122890,-4.169862,4.318407,-0.092768,59.662440,31.153680,-2.221401,-2.005754,2.992937,0.102457,54.072360,67.911840,-0.302513,0.702790,0.765133,-0.164093,47.188440,3.601440,-2.037537,-1.966954,2.832043,0.582805,40.859640,19.715760,-3.246034,1.629536,3.632097,0.325550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.476120,28.917630
1,2,1,0.2,1,1,B,NaN,B21,PASS,0.131004,34.731360,0.180384,-0.098375,0.205466,-0.538315,35.232084,47.018952,-0.193902,-2.289818,2.298013,0.291836,36.379044,35.250840,-0.276528,0.302931,0.410165,-0.777740,33.415308,25.587864,0.391235,-0.200779,0.439747,3.015208,34.709796,15.280344,0.912207,-2.746951,2.894454,-0.014227,44.360460,52.240104,0.258547,-2.868839,2.880466,-0.285167,45.030816,34.505640,-0.149472,-0.003543,0.149514,1.348017,42.264072,23.414040,1.081533,-3.113541,3.296036,0.162448,49.011264,15.162048,-0.001961,-3.164156,3.164157,0.916124,56.940408,27.356616,0.238109,-5.215412,5.220845,0.254324,59.646132,31.175424,0.724136,-4.409863,4.468922,-2.535089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.608456,34.173576,-1.191155,-0.280863,1.223820,-0.523442,63.013248,15.045912,-1.891850,0.234629,1.906344,-1.019133,73.043748,33.658920,-1.823416,-1.642523,2.454125,-0.951439,72.662940,55.100736,-2.084835,-1.968142,2.867075,0.745047,44.025228,44.280576,-1.397685,-3.069120,3.372391,0.446115,49.087080,27.905328,-1.148448,-3.674203,3.849507,-0.058290,60.453216,48.801744,1.204881,-4.125253,4.297610,-0.249277,59.646132,31

In [8]:
def preprocessing_tracking_data(df, frame):

  # period	frame	time	xPos	yPos	playerId	team	xPosMetrica	yPosMetrica	ballInPlay
  basic = df[df['frame'] == frame].copy()

  df_list = []
  frame_1_basic = list(basic[['frame', 'session', 'time']].iloc[0])

  # ball 전처리
  ball_basic = list(basic.filter(regex=f'ball').iloc[0]) + [0,0,0,0]
  ball_basic += [-1]
  ball_basic += ['Ball']
  ball_basic = frame_1_basic + ball_basic
  df_list.append(ball_basic)

  # player 전처리
  for i in range(1, 29):
    if i < 10:
      i_basic = list(basic.filter(regex=f'0{i}').iloc[0])
    else:
      i_basic = list(basic.filter(regex=f'{i}').iloc[0])
    if len(i_basic) == 0:
      i_basic = [0,0,0,0]
    i_basic += [i]
    if i <= 14 :
      i_basic += ['Home']
    else:
      i_basic += ['Away']
    frame_df = frame_1_basic + i_basic
    df_list.append(frame_df)

  df_basic = pd.DataFrame(df_list, columns = ['frame', 'period', 'time', 'xPos', 'yPos', 'xvel', 'yvel', 'speed', 'acel', 'playerId', 'team'])
  df_basic.dropna(subset=['xPos', 'yPos'], inplace=True)

  df_basic['xPosMetrica'] = df_basic['xPos']
  df_basic['yPosMetrica'] = df_basic['yPos']
  df_basic = df_basic[['period','frame','time','xPos','yPos','playerId','team','xPosMetrica','yPosMetrica']].copy()

  return df_basic

In [9]:
# BallInPlay 구하기
# episode 칼럼 활용하여 BallInPlay 구하기
def episode_to_ballinplay(df):
  epi_list = list(df['episode'])
  ball_list = []
  for i in epi_list:
    if i == 0:
      ball_list.append(0)
    else:
      ball_list.append(1)
  df['ballInPlay'] = ball_list
  return df

df_match_1 = episode_to_ballinplay(df_match_1)
df_match_2 = episode_to_ballinplay(df_match_2)
df_match_3 = episode_to_ballinplay(df_match_3)

display(df_match_1.head())
display(df_match_2.head())
display(df_match_3.head())

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,1,B,NaN,B19,PASS,0.100440,34.731360,0.333290,-0.035174,0.335140,-0.625875,35.259840,47.031840,-0.251670,-2.273060,2.286950,-0.059494,36.397080,35.181360,-0.245772,0.820850,0.856854,-2.339236,33.401160,25.580880,0.086596,0.185292,0.204529,3.577530,34.707960,15.308640,0.906415,-2.713272,2.860670,0.463751,44.381520,52.264080,0.211816,-2.904561,2.912274,-0.290737,45.033840,34.446960,0.105120,0.959609,0.965349,-0.175408,42.255000,23.436000,1.116675,-3.071419,3.268115,0.208952,49.019040,15.245280,0.056168,-3.032358,3.032878,1.306999,56.912760,27.345600,0.183449,-5.193607,5.196846,0.145989,59.662440,31.153680,1.015902,-4.618719,4.729125,-2.593574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.731495,34.172730,-1.339970,-0.019173,1.340108,-0.864164,63.064440,14.971680,-2.051766,0.765264,2.189834,-2.652163,73.070640,33.631200,-1.938117,-1.679287,2.564430,-1.044052,72.694800,55.062720,-2.074982,-1.893343,2.808968,0.341137,44.045640,44.298000,-1.353995,-3.043234,3.330852,0.230280,49.109760,27.870480,-1.115580,-3.675937,3.841488,0.074951,60.436800,48.798000,1.122890,-4.169862,4.318407,-0.092768,59.662440,31.153680,-2.221401,-2.005754,2.992937,0.102457,54.072360,67.911840,-0.302513,0.702790,0.765133,-0.164093,47.188440,3.601440,-2.037537,-1.966954,2.832043,0.582805,40.859640,19.715760,-3.246034,1.629536,3.632097,0.325550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.476120,28.917630,1
1,2,1,0.2,1,1,B,NaN,B21,PASS,0.131004,34.731360,0.180384,-0.098375,0.205466,-0.538315,35.232084,47.018952,-0.193902,-2.289818,2.298013,0.291836,36.379044,35.250840,-0.276528,0.302931,0.410165,-0.777740,33.415308,25.587864,0.391235,-0.200779,0.439747,3.015208,34.709796,15.280344,0.912207,-2.746951,2.894454,-0.014227,44.360460,52.240104,0.258547,-2.868839,2.880466,-0.285167,45.030816,34.505640,-0.149472,-0.003543,0.149514,1.348017,42.264072,23.414040,1.081533,-3.113541,3.296036,0.162448,49.011264,15.162048,-0.001961,-3.164156,3.164157,0.916124,56.940408,27.356616,0.238109,-5.215412,5.220845,0.254324,59.646132,31.175424,0.724136,-4.409863,4.468922,-2.535089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.608456,34.173576,-1.191155,-0.280863,1.223820,-0.523442,63.013248,15.045912,-1.891850,0.234629,1.906344,-1.019133,73.043748,33.658920,-1.823416,-1.642523,2.454125,-0.951439,72.662940,55.100736,-2.084835,-1.968142,2.867075,0.745047,44.025228,44.280576,-1.397685,-3.069120,3.372391,0.446115,49.087080,27.905328,-1.148448,-3.674203,3.849507,-0.058290,60.453216,48.801744,1.204881,-4.125253,4.297610,-0.249277

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,0,B,NaN,NaN,NaN,101.817000,36.297360,-0.236653,0.385690,0.452506,1.973214,69.996960,20.595600,0.089174,0.112731,0.143737,0.585066,73.17216,30.81816,0.0,-0.0,0.0,-0.011454,75.472560,40.036320,0.287884,0.039182,0.290538,-0.272804,69.495840,54.902160,-0.067458,-0.021741,0.070875,0.304460,25.206120,18.625680,-2.098519,-0.004036,2.098523,0.852876,65.09052,34.12584,0.0,0.0,0.0,0.048694,63.188640,38.172240,0.223802,0.098519,0.244527,0.341507,57.416040,54.060480,0.337413,0.456269,0.567476,0.145176,54.687960,21.698640,-0.105780,0.703339,0.711249,0.570755,54.209520,44.008560,-0.014387,0.637643,0.637805,1.059556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.315440,37.269360,0.000272,0.000453,0.000528,-0.032781,35.522280,58.929120,0.039522,0.030945,0.050195,-0.000788,33.192720,39.731040,0.015739,0.390371,0.390688,0.075031,34.813800,28.046880,-0.605064,0.131292,0.619145,-0.051233,39.4254,22.35528,0.0,0.0,0.0,0.024619,53.52912,65.78208,0.0,0.0,0.0,0.019074,42.996960,33.996960,-0.000143,0.763925,0.763925,0.318623,46.778040,22.574880,-0.019010,0.750436,0.750677,0.469109,53.880120,-1.629360,-0.156713,-0.203589,0.256919,-0.387868,54.19116,36.1044,0.0,0.0,0.0,0.063357,53.47836,26.08848,-0.0,-0.0,0.0,-0.008270,NaN,NaN,NaN,NaN,NaN,NaN,54.07803,36.27549,0
1,2,1,0.2,1,1,B,NaN,NaN,NaN,101.817000,36.297360,0.103679,-0.167974,0.197395,1.936080,69.996960,20.595600,-0.027428,-0.059382,0.065410,0.684876,73.17216,30.81816,0.0,-0.0,0.0,-0.004220,75.472560,40.036320,-0.020887,0.042941,0.047752,0.612635,69.495840,54.902160,0.043492,-0.025243,0.050287,0.327896,25.114644,18.626400,-2.170784,0.019561,2.170872,0.351186,65.09052,34.12584,0.0,0.0,0.0,0.002315,63.188640,38.172240,0.253622,0.132851,0.286310,0.172552,57.416040,54.060480,0.346861,0.470350,0.584416,0.082514,54.687960,21.698640,-0.121662,0.755567,0.765299,0.247627,54.209520,44.008560,-0.023526,0.724065,0.724447,0.455251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.315440,37.269360,0.000030,0.000050,0.000059,-0.012652,35.463528,58.929624,0.043751,0.021358,0.048686,-0.005431,33.192720,39.731040,0.023046,0.388525,0.389208,-0.102646,34.813800,28.046880,-0.600704,0.146169,0.618231,0.011239,39.4254,22.35528,0.0,0.0,0.0,0.002135,53.52912,65.78208,0.0,0.0,0.0,0.001324,42.996960,33.996960,0.002260,0.784030,0.784033,-0.023239,46.778040,22.574880,-0.019523,0.796150,0.796389,0.194995,53.880120,-1.629360,-0.150558,-0.153946,0.215330,-0.253593,54.19116,36.1044,0.0,0.0,0.0,0.002829,53.47836,26.08848,-0.0,-0.0,0.0,-0.002514,NaN,NaN,NaN,NaN,NaN,NaN,54.07803,36.27549,1
2,3,1,0.3,1,1,B,NaN,NaN,NaN,101.817000,36.297360,0.379209,-0.618254,0.725284,1.790562,69.996960,20.595600,-0.11

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,A15_x,A15_y,A15_vx,A15_vy,A15_speed,A15_accel,A16_x,A16_y,A16_vx,A16_vy,A16_speed,A16_accel,A17_x,A17_y,A17_vx,A17_vy,A17_speed,A17_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B29_x,B29_y,B29_vx,B29_vy,B29_speed,B29_accel,B30_x,B30_y,B30_vx,B30_vy,B30_speed,B30_accel,B31_x,B31_y,B31_vx,B31_vy,B31_speed,B31_accel,B32_x,B32_y,B32_vx,B32_vy,B32_speed,B32_accel,B33_x,B33_y,B33_vx,B33_vy,B33_speed,B33_accel,B34_x,B34_y,B34_vx,B34_vy,B34_speed,B34_accel,B35_x,B35_y,B35_vx,B35_vy,B35_speed,B35_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,0,A,NaN,NaN,NaN,91.49976,38.0556,0.0,0.0,0.0,0.0,70.459875,17.713350,-0.071801,-1.471252,1.473003,-0.971570,71.815950,33.517350,-0.534203,-0.151597,0.555297,0.472244,73.543545,42.525000,-0.444226,-0.495043,0.665134,-1.996977,67.372155,58.048560,-0.771250,0.148003,0.785323,-0.262673,54.522855,11.192220,-0.296109,-0.311382,0.429697,-1.138126,65.651175,32.400270,-0.319046,0.418088,0.525916,-0.659212,54.188190,32.740200,-0.084754,1.553790,1.556100,-2.972459,66.955140,43.588710,-0.268255,0.677334,0.728520,-1.573195,55.958580,55.531800,-0.480321,-0.553640,0.732956,-0.977324,54.593595,36.471690,0.219643,-0.595247,0.634478,-1.940730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7588,35.7876,0.0,0.0,0.0,0.0,36.966780,57.469950,0.078251,0.129162,0.151017,-0.308812,31.124385,49.928490,0.259957,-0.167389,0.309187,1.357368,32.878575,30.814110,-1.087167,-0.365654,1.147012,-1.503122,35.022915,16.113960,-0.768273,-0.786708,1.099615,-2.779477,47.995065,44.823600,-0.127491,-0.575637,0.589587,-1.351388,44.686890,38.015640,-0.080743,1.981077,1.982722,-2.928275,45.081090,34.518420,-0.255039,-0.555075,0.610862,-1.021945,44.456040,26.173350,-0.049030,-0.711380,0.713068,1.173747,50.811975,52.713180,1.222197,0.843206,1.484844,-0.766918,52.750170,26.180190,-0.536658,-0.602870,0.807127,-1.771434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.12339,35.08146,0
1,2,1,0.2,1,0,A,NaN,NaN,NaN,91.49976,38.0556,0.0,0.0,0.0,0.0,70.381224,17.500752,-0.106912,-1.335743,1.340015,-1.387549,71.776476,33.477840,-0.165739,-0.358251,0.394732,0.640638,73.509336,42.469560,-0.282102,-0.392744,0.483559,-1.442673,67.304736,58.038984,-0.744403,0.155629,0.760497,-0.298681,54.429732,11.030472,-0.247003,-0.242109,0.345872,-0.744949,65.623932,32.317488,-0.196424,0.396758,0.442718,-0.547116,54.129168,32.990688,-0.081520,1.263730,1.266356,-2.697232,66.932460,43.495560,-0.250726,0.518613,0.576041,-1.243867,55.914840,55.467936,-0.361413,-0.534276,0.645035,-0.693743,54.608688,36.378792,0.155680,-0.512655,0.535772,-0.473780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [10]:
# 전처리 및 Join

# tracking_df_list = [df_match_1, df_match_2, df_match_3]
# ball_df_list = [match_ball_1, match_ball_2, match_ball_3]

# for i in range(0,3):
#   dflist = []
#   for frame in tqdm(list(tracking_df_list[i]['frame'])):
#     dflist.append(preprocessing_tracking_data(tracking_df_list[i], frame))
#   df_accum = pd.concat(dflist)
#   df_prepro = pd.merge(df_accum, ball_df_list[i], how='inner', on=['frame', 'period', 'time'])

#   # parquet 변형
#   table = pa.Table.from_pandas(df_prepro)
#   pq.write_table(table, f'/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_{i+1}.parquet')

In [11]:
tracking_data_game_1 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_1.parquet").drop(labels='ballInPlay',axis=1)
tracking_data_game_1 = pd.merge(tracking_data_game_1, df_match_1[['frame', 'episode','ballInPlay']], how='inner', on=['frame'])

tracking_data_game_2 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_2.parquet").drop(labels='ballInPlay',axis=1)
tracking_data_game_2 = pd.merge(tracking_data_game_2, df_match_2[['frame', 'episode','ballInPlay']], how='inner', on=['frame'])

tracking_data_game_3 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/tracking_data_game_3.parquet").drop(labels='ballInPlay',axis=1)
tracking_data_game_3 = pd.merge(tracking_data_game_3, df_match_3[['frame', 'episode','ballInPlay']], how='inner', on=['frame'])

In [12]:
display(tracking_data_game_1.head())
display(tracking_data_game_2.head())
display(tracking_data_game_3.head())

,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,episode,ballInPlay
0,1.0,1.0,0.1,52.47612,28.91763,-1,Ball,52.47612,28.91763,1,1
1,1.0,1.0,0.1,35.25984,47.03184,1,Home,35.25984,47.03184,1,1
2,1.0,1.0,0.1,36.39708,35.18136,2,Home,36.39708,35.18136,1,1
3,1.0,1.0,0.1,33.40116,25.58088,3,Home,33.40116,25.58088,1,1
4,1.0,1.0,0.1,34.70796,15.30864,4,Home,34.70796,15.30864,1,1


,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,episode,ballInPlay
0,1.0,1.0,0.1,54.07803,36.27549,-1.0,Ball,54.07803,36.27549,0,0
1,1.0,1.0,0.1,69.99696,20.59560,1.0,Home,69.99696,20.59560,0,0
2,1.0,1.0,0.1,73.17216,30.81816,2.0,Home,73.17216,30.81816,0,0
3,1.0,1.0,0.1,75.47256,40.03632,3.0,Home,75.47256,40.03632,0,0
4,1.0,1.0,0.1,69.49584,54.90216,4.0,Home,69.49584,54.90216,0,0


,period,frame,time,xPos,yPos,playerId,team,xPosMetrica,yPosMetrica,episode,ballInPlay
0,1.0,1.0,0.1,54.123390,35.08146,-1,Ball,54.123390,35.08146,0,0
1,1.0,1.0,0.1,70.459875,17.71335,1,Home,70.459875,17.71335,0,0
2,1.0,1.0,0.1,71.815950,33.51735,2,Home,71.815950,33.51735,0,0
3,1.0,1.0,0.1,73.543545,42.52500,3,Home,73.543545,42.52500,0,0
4,1.0,1.0,0.1,67.372155,58.04856,4,Home,67.372155,58.04856,0,0


**0-2) Event 데이터 전처리**

In [53]:
# event 데|이터 전처리
# df_event_match_1 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_1.csv")
# df_event_match_2 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_2.csv")
# df_event_match_3 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/event_match_3.csv")

# def accurate_list_function(df):
#   accurate_1 = df[['type', 'eventName', 'subtype', 'subEventName', 'accurate']][df['accurate']==1].head(100)
#   accurate_list = list(dict.fromkeys(list(accurate_1['eventName'].unique())+ list(accurate_1['subtype'].unique())+ list(accurate_1['subEventName'].unique())))
#   accurate_list_2 = []
#   for i in accurate_list:
#     accurate_list_2.append(i.upper())
#   return accurate_list_2


# def event_preprocessing(df, game, df_event_game_1):

#   game = game

#   df['playerId'] = df['from'].str[1:].apply(pd.to_numeric)
#   df['toPlayerId'] = df['to'].str[1:].apply(pd.to_numeric)
#   df['id'] = [i for i in range(0, len(df))]
#   df['matchId'] = game
#   df['eventName'] = df['type']
#   df['subEventName'] = df['subtype']
#   # eventName 변환 & accurate 계산
#   df.replace('CHALLENGE', 'Duel', inplace=True)

#   accurate_list = [0]
#   home = [i for i in range(1, 15)]
#   away = [i for i in range(15, 28)]
#   accurate_list_2 = accurate_list_function(df_event_game_1)
#   for i in tqdm(range(1, len(df))):
#     i_series = df[['type','subtype','eventName','subEventName']].iloc[i]
#     if df.loc[i-1, 'eventName'] == 'Set piece':
#       df.loc[i, 'eventName'] = 'Set piece'

#     accurate = 1
#     pass_count = 0

#     for a in i_series.to_list():
#       if a == 'PASS':
#         pass_count += 1
#       if a not in accurate_list_2:
#         accurate = 0
#         break
#     if pass_count == 0:
#       accurate = 0

#     if (df.loc[i, 'playerId'] in home) & (df.loc[i, 'toPlayerId'] in home):
#       accurate = 1
#     elif (df.loc[i, 'playerId'] in away) & (df.loc[i, 'toPlayerId'] in away):
#       accurate = 1
#     else:
#       accurate = 0


#     accurate_list.append(accurate)
#   df['accurate'] = accurate_list

#   df.rename(columns={'start_frame':'startFrame', 'end_frame':'endFrame'}, inplace=True)
#   df['startFrame'] = df['startFrame'].apply(pd.to_numeric)
#   df['endFrame'] = df['endFrame'].apply(pd.to_numeric)

#   return df

# df_event_pro_1 = event_preprocessing(df_event_match_1, 1, df_event_game_1)
# df_event_pro_2 = event_preprocessing(df_event_match_2, 2, df_event_game_1)
# df_event_pro_3 = event_preprocessing(df_event_match_3, 3, df_event_game_1)

In [38]:
# event 데이터를 metrica_data 폴더에 parquet 형식으로 저장

# a = 1
# for i in [df_event_pro_1, df_event_pro_2, df_event_pro_3]:
#   table = pa.Table.from_pandas(i)
#   pq.write_table(table, f'/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_{a}.parquet')
#   a += 1

In [39]:
# # 양식 맞춘 후, io_help/td_help 이용해서 데이터 변환
# game=1
# df_track = io_help.read_tracking_data(game=game, clean=True)
# df_track = td_help.add_player_velocities(df_track)
# display(df_track.head())

# # event의 경우, success 패스를 구하기 위한 최소한의 칼럼만을 구현했으므로, 전처리 그대로 활용
# df_event_game_1 = pd.read_parquet("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_1.parquet")

In [79]:
df_event_game_1 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_1.csv")
df_event_game_2 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_2.csv")
df_event_game_3 = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/event_data_game_3.csv")

all_match = pd.read_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/metrica_data/all-match.csv")
all_match.head()

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,subEventName,accurate,freeze_frame,value_label,game_id,pass_id
0,Away,SET PIECE,KICK OFF,1,1,0.1,0,0.0,B19,NaN,NaN,NaN,NaN,NaN,1,0,0,2,Set piece,Kick off,0,{},0,1,0
1,Away,PASS,PASS,1,1,0.1,2,0.2,B19,B21,0.45,0.39,0.55,0.43,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,1
2,Away,PASS,PASS,1,2,0.2,7,0.7,B21,B15,0.55,0.43,0.58,0.21,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,2
3,Away,PASS,PASS,1,18,1.8,25,2.5,B15,B19,0.55,0.19,0.45,0.31,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,3
4,Away,PASS,PASS,1,31,3.1,39,3.9,B19,B21,0.45,0.32,0.49,0.47,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,4


In [80]:
print(f'df_event_game_1 :  {len(df_event_game_1)}')
print(f'df_event_game_2 :  {len(df_event_game_2)}')
print(f'df_event_game_3 :  {len(df_event_game_3)}')

df_event_game_1 = all_match[all_match['game_id'] == 1]
df_event_game_2 = all_match[all_match['game_id'] == 2]
df_event_game_3 = all_match[all_match['game_id'] == 3]

print(f'df_event_game_1 :  {len(df_event_game_1)}')
print(f'df_event_game_2 :  {len(df_event_game_2)}')
print(f'df_event_game_3 :  {len(df_event_game_3)}')

df_event_game_1 :  1745
df_event_game_2 :  1935
df_event_game_3 :  3620
df_event_game_1 :  1745
df_event_game_2 :  1935
df_event_game_3 :  3620


**1) Success Intended Receiver 예측 및 결과 비교**

**1-1) Game 1 Intended Receiver 예측 (정확도: 99.87%)**

In [81]:
# 양식 맞춘 후, 데이터 변환
display(df_match_1.head())
display(df_event_game_1.head())

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,1,B,NaN,B19,PASS,0.100440,34.731360,0.333290,-0.035174,0.335140,-0.625875,35.259840,47.031840,-0.251670,-2.273060,2.286950,-0.059494,36.397080,35.181360,-0.245772,0.820850,0.856854,-2.339236,33.401160,25.580880,0.086596,0.185292,0.204529,3.577530,34.707960,15.308640,0.906415,-2.713272,2.860670,0.463751,44.381520,52.264080,0.211816,-2.904561,2.912274,-0.290737,45.033840,34.446960,0.105120,0.959609,0.965349,-0.175408,42.255000,23.436000,1.116675,-3.071419,3.268115,0.208952,49.019040,15.245280,0.056168,-3.032358,3.032878,1.306999,56.912760,27.345600,0.183449,-5.193607,5.196846,0.145989,59.662440,31.153680,1.015902,-4.618719,4.729125,-2.593574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.731495,34.172730,-1.339970,-0.019173,1.340108,-0.864164,63.064440,14.971680,-2.051766,0.765264,2.189834,-2.652163,73.070640,33.631200,-1.938117,-1.679287,2.564430,-1.044052,72.694800,55.062720,-2.074982,-1.893343,2.808968,0.341137,44.045640,44.298000,-1.353995,-3.043234,3.330852,0.230280,49.109760,27.870480,-1.115580,-3.675937,3.841488,0.074951,60.436800,48.798000,1.122890,-4.169862,4.318407,-0.092768,59.662440,31.153680,-2.221401,-2.005754,2.992937,0.102457,54.072360,67.911840,-0.302513,0.702790,0.765133,-0.164093,47.188440,3.601440,-2.037537,-1.966954,2.832043,0.582805,40.859640,19.715760,-3.246034,1.629536,3.632097,0.325550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.476120,28.917630,1
1,2,1,0.2,1,1,B,NaN,B21,PASS,0.131004,34.731360,0.180384,-0.098375,0.205466,-0.538315,35.232084,47.018952,-0.193902,-2.289818,2.298013,0.291836,36.379044,35.250840,-0.276528,0.302931,0.410165,-0.777740,33.415308,25.587864,0.391235,-0.200779,0.439747,3.015208,34.709796,15.280344,0.912207,-2.746951,2.894454,-0.014227,44.360460,52.240104,0.258547,-2.868839,2.880466,-0.285167,45.030816,34.505640,-0.149472,-0.003543,0.149514,1.348017,42.264072,23.414040,1.081533,-3.113541,3.296036,0.162448,49.011264,15.162048,-0.001961,-3.164156,3.164157,0.916124,56.940408,27.356616,0.238109,-5.215412,5.220845,0.254324,59.646132,31.175424,0.724136,-4.409863,4.468922,-2.535089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.608456,34.173576,-1.191155,-0.280863,1.223820,-0.523442,63.013248,15.045912,-1.891850,0.234629,1.906344,-1.019133,73.043748,33.658920,-1.823416,-1.642523,2.454125,-0.951439,72.662940,55.100736,-2.084835,-1.968142,2.867075,0.745047,44.025228,44.280576,-1.397685,-3.069120,3.372391,0.446115,49.087080,27.905328,-1.148448,-3.674203,3.849507,-0.058290,60.453216,48.801744,1.204881,-4.125253,4.297610,-0.249277

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,subEventName,accurate,freeze_frame,value_label,game_id,pass_id
0,Away,SET PIECE,KICK OFF,1,1,0.1,0,0.0,B19,NaN,NaN,NaN,NaN,NaN,1,0,0,2,Set piece,Kick off,0,{},0,1,0
1,Away,PASS,PASS,1,1,0.1,2,0.2,B19,B21,0.45,0.39,0.55,0.43,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,1
2,Away,PASS,PASS,1,2,0.2,7,0.7,B21,B15,0.55,0.43,0.58,0.21,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,2
3,Away,PASS,PASS,1,18,1.8,25,2.5,B15,B19,0.55,0.19,0.45,0.31,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,3
4,Away,PASS,PASS,1,31,3.1,39,3.9,B19,B21,0.45,0.32,0.49,0.47,1,0,0,2,Pass,Pass,1,"{'B25': {'teammate': True, 'actor': False, 'ba...",0,1,4


In [86]:
# get all passes
df_pass = df_event_game_1[
                (df_event_game_1["eventName"] == "Pass")
                & (df_event_game_1["start_frame"] < df_event_game_1["end_frame"])
                & (df_event_game_1["start_frame"] >= 2)
            ].copy()


# only keep relevant columns
df_pass = df_pass[["team", "start_frame", "end_frame", "eventName", "subEventName", "from", "to", "freeze_frame" ,"accurate", "pass_id"]].copy()

# attach the ball position at the start and end frame
df_ball = df_match_1[["frame", "ball_x", "ball_y", "episode", "ballInPlay",]].copy()

df_ball.columns = ["start_frame", "xPosStart", "yPosStart", "episodeStart", "ballInPlayStart"]
df_pass = pd.merge(df_pass, df_ball, how="left")

df_ball.columns = ["end_frame", "xPosEnd", "yPosEnd", "episodeEnd", "ballInPlayEnd"]
df_pass = pd.merge(df_pass, df_ball, how="left")
df_pass = df_pass[df_pass["xPosEnd"].notnull()].copy()

# only keep passes for which the ball was in play at the beginning of the pass (i.e. exclude throw-ins)
df_pass = df_pass[df_pass["ballInPlayStart"] == 1].copy()
df_pass.reset_index(inplace=True, drop=True)

# only keep accurate passes
df_pass_acc = df_pass[df_pass["accurate"] == 1].copy()

display(df_pass_acc.head(20))
print(f'정확한 패스의 개수 : {len(df_pass_acc)}/{len(df_pass)}')

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd
0,Away,2,7,Pass,Pass,B21,B15,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,2,59.299344,30.117744,1,1,61.988436,15.574464,1,1
1,Away,18,25,Pass,Pass,B15,B19,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,3,59.431968,13.655592,1,1,48.856716,21.991320,1,1
2,Away,31,39,Pass,Pass,B19,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,4,48.902292,22.805712,1,1,52.426980,34.024032,1,1
3,Away,77,87,Pass,Pass,B21,B22,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,5,43.330140,52.546752,1,1,34.839396,69.503976,1,1
4,Away,112,122,Pass,Pass,B22,B17,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,6,41.639076,69.283512,1,1,53.252208,70.278192,1,1
8,Home,200,215,Pass,Head,A02,A09,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,15,39.545172,48.012696,1,1,56.666412,42.483312,1,1
9,Home,215,223,Pass,Pass,A09,A10,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,16,56.666412,42.483312,1,1,54.236952,46.710576,1,1
11,Away,306,314,Pass,Pass,B16,B19,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,19,62.242668,19.353528,1,1,55.535112,23.789088,1,1
12,Away,314,322,Pass,Pass,B19,B20,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,20,55.535112,23.789088,1,1,60.800328,33.627024,1,1
13,Away,334,353,Pass,Pass,B20,B22,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,21,57.525120,37.922688,1,1,47.410812,65.849472,1,1


정확한 패스의 개수 : 796/938


In [87]:
# Intended Receiver 예측

intended_list = []
frame_dist = []
frame_receiver = []
frame_ball = []
frame_angle = []

# match dataframe 에서 선수들 위치 정보 추출 함수
def extract_player_pos(df, frame):

  basic = df[df['frame'] == frame]

  # player 전처리
  player_df = []
  for i in range(1, 29):
    if i < 10:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'0{i}').iloc[0])[0:2]
      i_basic += ['Home']
    else:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'{i}').iloc[0])[0:2]
      if i <= 14 :
        i_basic += ['Home']
      else:
        i_basic += ['Away']
    player_df.append(i_basic)
  player_df = pd.DataFrame(player_df, columns=['playerId', 'xPos', 'yPos', 'team']).dropna(axis=0)

  return player_df


for idx, action in tqdm(df_pass_acc.iterrows()):
  # get coordinates of the pass start location, interception point
  # and each potential receiver
  df_start_frame = extract_player_pos(df_match_1, action.start_frame)
  receiver_coo = np.array(
      [
          (o["xPos"], o["yPos"])
          for index, o in df_start_frame.iterrows()
      ]
  )

  len_home = len(df_start_frame[df_start_frame['team'] == 'Home'])
  if action['team'] == 'Home':
    receiver_coo = receiver_coo[:len_home]
  else:
    receiver_coo = receiver_coo[len_home:]

  ball_coo = np.array([action.xPosStart, action.yPosStart])
  interception_coo = np.array([action.xPosEnd, action.yPosEnd])

  # compute the distance between the location where the ball was
  # intercepted and each potential receiver
  dist = np.sqrt(
                (receiver_coo[:, 0] - interception_coo[0]) ** 2
                + (receiver_coo[:, 1] - interception_coo[1]) ** 2
            )

  frame_dist.append(dist)
  frame_ball.append(ball_coo)
  frame_receiver.append(receiver_coo)

  # compute the angle between each potential receiver and the passing line
  a = interception_coo - ball_coo
  b = receiver_coo - ball_coo
  angle = np.arccos(
                np.clip(
                    np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1)), -1, 1
                )
            )


  # if not players are in 20 degrees of the pass line, the intended
  # receiver was probably not in the freeeze frame
  if np.amin(angle) > 0.35:
    intended_list.append(0)
    continue
  # only consider players in 20 degrees of the pass line
  too_wide = np.where(angle > 0.35)[0]
  dist[too_wide] = np.inf

  # find the most likely intended receiver
  # TODO: you could play around with the weight given to the distance
  # and angle here
  exp_receiver = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))

  # start_frame 에서 선수들 모두 리스트업
  # df_start_frame = extract_player_pos(df_match_1, action.start_frame)
  id_list = list(df_start_frame["playerId"])

  # Home, Away 구분
  if action['from'][0] == "A" :
    id = id_list[:len_home]
  else:
    id = id_list[len_home:]

  try:
    intended_list.append(id[exp_receiver])
  except:
    print(f'idx : {idx}')
    print(f'len_home : {len_home}')
    print(f'receiver 수 : {len(receiver_coo)}')
    print(f'총 선수 수 : {len(id_list)}')
    print(f'타겟 팀 선수 수 : {len(id)}')
    print(f'오차 : {len(receiver_coo) - len(id)}')

df_pass_acc['intended_receiver'] = intended_list

796it [00:17, 45.92it/s]


In [88]:
# 기존 데이터와의 Accuracy 실험
accurate_count = 0

err_str_frame = []
err_end_frame = []
err_idx = []
for idx, row in df_pass_acc.iterrows():
  toplayerId = int(row['to'][1:])
  if toplayerId == row['intended_receiver']:
    accurate_count += 1
  else:
    err_str_frame.append(row['start_frame'])
    err_end_frame.append(row['end_frame'])
    err_idx.append(idx)

accuracy_rate = accurate_count / len(df_pass_acc) * 100

print(f'Game 1에 대한 intended receiver 정확도 : {accuracy_rate}')

Game 1에 대한 intended receiver 정확도 : 80.65326633165829


In [89]:
match_1_err = df_pass_acc[df_pass_acc['start_frame'].isin(err_str_frame)]
display(match_1_err.head(10))

match_1_err.to_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match_1_err.csv")

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd,intended_receiver
18,Home,491,507,Pass,Pass,A10,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,27,49.157496,48.760848,1,1,60.400836,24.836904,1,1,0
21,Home,906,916,Pass,Cross,A10,A09,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,33,96.281460,10.170000,2,1,99.138924,33.176304,2,1,8
39,Home,2251,2266,Pass,Pass,A01,A02,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,74,16.169976,41.650632,5,1,19.548324,37.925496,5,1,8
41,Home,2305,2324,Pass,Pass,A08,A10,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,78,42.539688,20.885328,5,1,64.694160,6.403392,5,1,8
43,Away,2746,2755,Pass,Pass,B21,B20,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,86,58.183488,21.729888,6,1,58.614732,26.668656,6,1,22
47,Home,2817,2860,Pass,Pass,A03,A04,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,92,11.128428,30.432096,6,1,3.459996,2.261664,6,1,0
48,Away,2882,2907,Pass,Pass,B18,B21,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,96,14.712732,1.278432,6,1,21.312180,7.783560,6,1,20
58,Away,3686,3702,Pass,Pass,B23,B18,"{'B25': {'teammate': True, 'actor': False, 'ba...",1,129,6.466537,0.205429,7,1,23.110596,3.936024,7,1,15
62,Home,3840,3852,Pass,Deep ball,A10,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,134,93.619584,20.613312,7,1,103.709808,27.326664,7,1,0
78,Home,5418,5450,Pass,Goal kick,A11,A05,"{'A11': {'teammate': True, 'actor': True, 'bal...",1,168,4.952016,31.984992,11,1,53.597052,61.592544,11,1,1


**1-2) Game 2 Intended Receiver 예측 (정확도 : 99.68%)**

In [ ]:
# 양식 맞춘 후, 데이터 변환
display(df_match_2.head())
display(df_event_game_2.head())

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B15_x,B15_y,B15_vx,B15_vy,B15_speed,B15_accel,B16_x,B16_y,B16_vx,B16_vy,B16_speed,B16_accel,B17_x,B17_y,B17_vx,B17_vy,B17_speed,B17_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,0,B,NaN,NaN,NaN,101.817000,36.297360,-0.236653,0.385690,0.452506,1.973214,69.996960,20.595600,0.089174,0.112731,0.143737,0.585066,73.17216,30.81816,0.0,-0.0,0.0,-0.011454,75.472560,40.036320,0.287884,0.039182,0.290538,-0.272804,69.495840,54.902160,-0.067458,-0.021741,0.070875,0.304460,25.206120,18.625680,-2.098519,-0.004036,2.098523,0.852876,65.09052,34.12584,0.0,0.0,0.0,0.048694,63.188640,38.172240,0.223802,0.098519,0.244527,0.341507,57.416040,54.060480,0.337413,0.456269,0.567476,0.145176,54.687960,21.698640,-0.105780,0.703339,0.711249,0.570755,54.209520,44.008560,-0.014387,0.637643,0.637805,1.059556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.315440,37.269360,0.000272,0.000453,0.000528,-0.032781,35.522280,58.929120,0.039522,0.030945,0.050195,-0.000788,33.192720,39.731040,0.015739,0.390371,0.390688,0.075031,34.813800,28.046880,-0.605064,0.131292,0.619145,-0.051233,39.4254,22.35528,0.0,0.0,0.0,0.024619,53.52912,65.78208,0.0,0.0,0.0,0.019074,42.996960,33.996960,-0.000143,0.763925,0.763925,0.318623,46.778040,22.574880,-0.019010,0.750436,0.750677,0.469109,53.880120,-1.629360,-0.156713,-0.203589,0.256919,-0.387868,54.19116,36.1044,0.0,0.0,0.0,0.063357,53.47836,26.08848,-0.0,-0.0,0.0,-0.008270,NaN,NaN,NaN,NaN,NaN,NaN,54.07803,36.27549,0
1,2,1,0.2,1,1,B,NaN,NaN,NaN,101.817000,36.297360,0.103679,-0.167974,0.197395,1.936080,69.996960,20.595600,-0.027428,-0.059382,0.065410,0.684876,73.17216,30.81816,0.0,-0.0,0.0,-0.004220,75.472560,40.036320,-0.020887,0.042941,0.047752,0.612635,69.495840,54.902160,0.043492,-0.025243,0.050287,0.327896,25.114644,18.626400,-2.170784,0.019561,2.170872,0.351186,65.09052,34.12584,0.0,0.0,0.0,0.002315,63.188640,38.172240,0.253622,0.132851,0.286310,0.172552,57.416040,54.060480,0.346861,0.470350,0.584416,0.082514,54.687960,21.698640,-0.121662,0.755567,0.765299,0.247627,54.209520,44.008560,-0.023526,0.724065,0.724447,0.455251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.315440,37.269360,0.000030,0.000050,0.000059,-0.012652,35.463528,58.929624,0.043751,0.021358,0.048686,-0.005431,33.192720,39.731040,0.023046,0.388525,0.389208,-0.102646,34.813800,28.046880,-0.600704,0.146169,0.618231,0.011239,39.4254,22.35528,0.0,0.0,0.0,0.002135,53.52912,65.78208,0.0,0.0,0.0,0.001324,42.996960,33.996960,0.002260,0.784030,0.784033,-0.023239,46.778040,22.574880,-0.019523,0.796150,0.796389,0.194995,53.880120,-1.629360,-0.150558,-0.153946,0.215330,-0.253593,54.19116,36.1044,0.0,0.0,0.0,0.002829,53.47836,26.08848,-0.0,-0.0,0.0,-0.002514,NaN,NaN,NaN,NaN,NaN,NaN,54.07803,36.27549,1
2,3,1,0.3,1,1,B,NaN,NaN,NaN,101.817000,36.297360,0.379209,-0.618254,0.725284,1.790562,69.996960,20.595600,-0.11

,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,freeze_frame,goal,ownGoal,teamId,eventName,subEventName,accurate
0,Away,SET PIECE,KICK OFF,1,21,2.1,21,2.1,B23,NaN,NaN,NaN,NaN,NaN,1,"[{'teammate': True, 'actor': False, 'ball': Fa...",0,0,2,Set piece,Kick off,0
1,Away,PASS,PASS,1,21,2.1,35,3.5,B23,B20,0.50,0.50,0.40,0.51,1,"[{'teammate': True, 'actor': False, 'ball': Fa...",0,0,2,Pass,Pass,1
2,Away,PASS,PASS,1,59,5.9,75,7.5,B20,B18,0.43,0.50,0.44,0.22,1,"[{'teammate': True, 'actor': False, 'ball': Fa...",0,0,2,Pass,Pass,1
3,Away,PASS,PASS,1,100,10.0,114,11.4,B18,B17,0.47,0.19,0.31,0.28,1,"[{'teammate': True, 'actor': False, 'ball': Fa...",0,0,2,Pass,Pass,1
4,Away,PASS,PASS,1,127,12.7,139,13.9,B17,B16,0.29,0.32,0.26,0.58,1,"[{'teammate': True, 'actor': False, 'ball': Fa...",0,0,2,Pass,Pass,1


In [90]:
# get all passes
df_pass = df_event_game_2[
                (df_event_game_2["eventName"] == "Pass")
                & (df_event_game_2["start_frame"] < df_event_game_2["end_frame"])
                & (df_event_game_2["start_frame"] >= 2)
            ].copy()


# only keep relevant columns
df_pass = df_pass[["team", "start_frame", "end_frame", "eventName", "subEventName", "from", "to", "freeze_frame" ,"accurate", "pass_id"]].copy()

# attach the ball position at the start and end frame
df_ball = df_match_2[["frame", "ball_x", "ball_y", "episode", "ballInPlay",]].copy()

df_ball.columns = ["start_frame", "xPosStart", "yPosStart", "episodeStart", "ballInPlayStart"]
df_pass = pd.merge(df_pass, df_ball, how="left")

df_ball.columns = ["end_frame", "xPosEnd", "yPosEnd", "episodeEnd", "ballInPlayEnd"]
df_pass = pd.merge(df_pass, df_ball, how="left")
df_pass = df_pass[df_pass["xPosEnd"].notnull()].copy()


# only keep passes for which the ball was in play at the beginning of the pass (i.e. exclude throw-ins)
df_pass = df_pass[df_pass["ballInPlayStart"] == 1].copy()
df_pass.reset_index(inplace=True, drop=True)

# str to dictionary
import ast
df_pass['freeze_frame'] = df_pass['freeze_frame'].apply(ast.literal_eval)

# only keep accurate passes
df_pass_acc = df_pass[df_pass["accurate"] == 1].copy()

display(df_pass_acc.head(20))
print(f'정확한 패스의 개수 : {len(df_pass_acc)}/{len(df_pass)}')

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd
0,Away,21,35,Pass,Pass,B23,B20,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1746,54.078030,36.275490,1,1,43.991748,36.590472,1,1
1,Away,59,75,Pass,Pass,B20,B18,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1747,45.921816,35.845848,1,1,47.858904,16.206408,1,1
2,Away,100,114,Pass,Pass,B18,B17,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1748,49.789728,13.635576,1,1,33.705180,20.124144,1,1
3,Away,127,139,Pass,Pass,B17,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1749,31.709448,22.819392,1,1,28.402488,41.246280,1,1
4,Away,158,170,Pass,Pass,B16,B15,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1750,27.789588,47.106144,1,1,28.991520,65.218320,1,1
6,Away,269,289,Pass,Pass,B15,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1757,53.670182,72.601106,1,1,35.746272,66.066336,1,1
7,Away,308,324,Pass,Pass,B16,B17,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1758,30.944268,60.135984,1,1,24.500556,38.085552,1,1
8,Away,342,370,Pass,Pass,B17,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1759,23.518404,36.627120,1,1,24.389640,61.682472,1,1
9,Away,370,379,Pass,Pass,B16,B15,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1760,24.389640,61.682472,1,1,29.707884,70.878456,1,1
10,Away,383,407,Pass,Pass,B15,B16,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1761,29.809836,71.293104,1,1,16.176240,64.632024,1,1


정확한 패스의 개수 : 964/1097


In [91]:
# Intended Receiver 예측

intended_list = []
frame_dist = []
frame_receiver = []
frame_ball = []
frame_angle = []

# match dataframe 에서 선수들 위치 정보 추출 함수
def extract_player_pos(df, frame):

  basic = df[df['frame'] == frame]

  # player 전처리
  player_df = []
  for i in range(1, 29):
    if i < 10:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'0{i}').iloc[0])[0:2]
      i_basic += ['Home']
    else:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'{i}').iloc[0])[0:2]
      if i <= 14 :
        i_basic += ['Home']
      else:
        i_basic += ['Away']
    player_df.append(i_basic)
  player_df = pd.DataFrame(player_df, columns=['playerId', 'xPos', 'yPos', 'team']).dropna(axis=0)

  return player_df


for idx, action in tqdm(df_pass_acc.iterrows()):
  # get coordinates of the pass start location, interception point
  # and each potential receiver
  df_start_frame = extract_player_pos(df_match_2, action.start_frame)
  receiver_coo = np.array(
      [
          (o["xPos"], o["yPos"])
          for index, o in df_start_frame.iterrows()
      ]
  )

  len_home = len(df_start_frame[df_start_frame['team'] == 'Home'])
  if action['team'] == 'Home':
    receiver_coo = receiver_coo[:len_home]
  else:
    receiver_coo = receiver_coo[len_home:]

  ball_coo = np.array([action.xPosStart, action.yPosStart])
  interception_coo = np.array([action.xPosEnd, action.yPosEnd])

  # compute the distance between the location where the ball was
  # intercepted and each potential receiver
  dist = np.sqrt(
                (receiver_coo[:, 0] - interception_coo[0]) ** 2
                + (receiver_coo[:, 1] - interception_coo[1]) ** 2
            )

  frame_dist.append(dist)
  frame_ball.append(ball_coo)
  frame_receiver.append(receiver_coo)

  # compute the angle between each potential receiver and the passing line
  a = interception_coo - ball_coo
  b = receiver_coo - ball_coo
  angle = np.arccos(
                np.clip(
                    np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1)), -1, 1
                )
            )


  # if not players are in 20 degrees of the pass line, the intended
  # receiver was probably not in the freeeze frame
  if np.amin(angle) > 0.35:
    intended_list.append(0)
    continue
  # only consider players in 20 degrees of the pass line
  too_wide = np.where(angle > 0.35)[0]
  dist[too_wide] = np.inf

  # find the most likely intended receiver
  # TODO: you could play around with the weight given to the distance
  # and angle here
  exp_receiver = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))

  # start_frame 에서 선수들 모두 리스트업
  id_list = list(df_start_frame["playerId"])

  # Home, Away 구분
  if action['from'][0] == "A" :
    id = id_list[:len_home]
  else:
    id = id_list[len_home:]

  try:
    intended_list.append(id[exp_receiver])
  except:
    print(f'idx : {idx}')
    print(f'len_home : {len_home}')
    print(f'receiver 수 : {len(receiver_coo)}')
    print(f'총 선수 수 : {len(id_list)}')
    print(f'타겟 팀 선수 수 : {len(id)}')
    print(f'오차 : {len(receiver_coo) - len(id)}')

df_pass_acc['intended_receiver'] = intended_list

964it [00:25, 37.96it/s]


In [92]:
# 기존 데이터와의 Accuracy 실험
accurate_count = 0

err_str_frame = []
err_end_frame = []
err_idx = []
for idx, row in df_pass_acc.iterrows():
  toplayerId = int(row['to'][1:])
  if toplayerId == row['intended_receiver']:
    accurate_count += 1
  else:
    err_str_frame.append(row['start_frame'])
    err_end_frame.append(row['end_frame'])
    err_idx.append(idx)

accuracy_rate = accurate_count / len(df_pass_acc) * 100

print(f'Game 2에 대한 intended receiver 정확도 : {accuracy_rate}')

Game 2에 대한 intended receiver 정확도 : 82.88381742738589


In [93]:
match_2_err = df_pass_acc[df_pass_acc['start_frame'].isin(err_str_frame)]
display(match_2_err.head(10))

match_2_err.to_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match_2_err.csv")

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd,intended_receiver
15,Away,516,528,Pass,Pass,B18,B24,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1770,52.782624,12.874464,1,1,66.433716,13.142448,1,1,0
36,Home,1084,1094,Pass,Pass,A01,A05,"{'A01': {'teammate': True, 'actor': True, 'bal...",1,1801,57.575880,4.545360,1,1,64.591884,9.452592,1,1,11
46,Home,1442,1457,Pass,Pass,A09,A06,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1814,40.554864,6.900120,1,1,45.329652,19.958616,1,1,4
50,Home,1532,1550,Pass,Pass,A08,A07,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1818,29.966112,67.769136,1,1,16.153236,67.189176,1,1,0
56,Home,1705,1717,Pass,Pass,A05,A09,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1824,27.851580,21.455640,1,1,17.244684,20.147544,1,1,0
57,Home,1717,1734,Pass,Pass,A09,A01,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1825,17.244684,20.147544,1,1,16.550676,8.578728,1,1,0
60,Away,1983,2017,Pass,Pass,B25,B23,"{'A01': {'teammate': False, 'actor': False, 'b...",1,1830,3.160220,35.961439,2,1,56.334204,20.634336,2,1,17
65,Home,2160,2170,Pass,Pass,A10,A07,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1839,30.942000,46.717200,2,1,31.213080,55.273896,2,1,8
66,Home,2179,2193,Pass,Pass,A07,A08,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1840,28.151712,56.321424,2,1,26.446284,66.038040,2,1,0
72,Home,2298,2306,Pass,Pass,A07,A09,"{'A01': {'teammate': True, 'actor': False, 'ba...",1,1846,21.971196,71.128296,2,1,20.638800,61.541640,2,1,10


**1-5) Game 3 Intended Receiver 예측 (정확도 : 88.22%)**

In [96]:
# 양식 맞춘 후, 데이터 변환
# match3의 event 데이터는 A팀이 Away로 표시되어 있으므로, 이에 대한 전처리 진행
display(df_match_3.head())

df_event_game_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)
display(df_event_game_3.head())

,frame,session,time,phase,episode,team_poss,player_poss,event_player,event_type,A11_x,A11_y,A11_vx,A11_vy,A11_speed,A11_accel,A01_x,A01_y,A01_vx,A01_vy,A01_speed,A01_accel,A02_x,A02_y,A02_vx,A02_vy,A02_speed,A02_accel,A03_x,A03_y,A03_vx,A03_vy,A03_speed,A03_accel,A04_x,A04_y,A04_vx,A04_vy,A04_speed,A04_accel,A05_x,A05_y,A05_vx,A05_vy,A05_speed,A05_accel,A06_x,A06_y,A06_vx,A06_vy,A06_speed,A06_accel,A07_x,A07_y,A07_vx,A07_vy,A07_speed,A07_accel,A08_x,A08_y,A08_vx,A08_vy,A08_speed,A08_accel,A09_x,A09_y,A09_vx,A09_vy,A09_speed,A09_accel,A10_x,A10_y,A10_vx,A10_vy,A10_speed,A10_accel,A12_x,A12_y,A12_vx,A12_vy,A12_speed,A12_accel,A13_x,A13_y,A13_vx,A13_vy,A13_speed,A13_accel,A14_x,A14_y,A14_vx,A14_vy,A14_speed,A14_accel,A15_x,A15_y,A15_vx,A15_vy,A15_speed,A15_accel,A16_x,A16_y,A16_vx,A16_vy,A16_speed,A16_accel,A17_x,A17_y,A17_vx,A17_vy,A17_speed,A17_accel,B28_x,B28_y,B28_vx,B28_vy,B28_speed,B28_accel,B18_x,B18_y,B18_vx,B18_vy,B18_speed,B18_accel,B19_x,B19_y,B19_vx,B19_vy,B19_speed,B19_accel,B20_x,B20_y,B20_vx,B20_vy,B20_speed,B20_accel,B21_x,B21_y,B21_vx,B21_vy,B21_speed,B21_accel,B22_x,B22_y,B22_vx,B22_vy,B22_speed,B22_accel,B23_x,B23_y,B23_vx,B23_vy,B23_speed,B23_accel,B24_x,B24_y,B24_vx,B24_vy,B24_speed,B24_accel,B25_x,B25_y,B25_vx,B25_vy,B25_speed,B25_accel,B26_x,B26_y,B26_vx,B26_vy,B26_speed,B26_accel,B27_x,B27_y,B27_vx,B27_vy,B27_speed,B27_accel,B29_x,B29_y,B29_vx,B29_vy,B29_speed,B29_accel,B30_x,B30_y,B30_vx,B30_vy,B30_speed,B30_accel,B31_x,B31_y,B31_vx,B31_vy,B31_speed,B31_accel,B32_x,B32_y,B32_vx,B32_vy,B32_speed,B32_accel,B33_x,B33_y,B33_vx,B33_vy,B33_speed,B33_accel,B34_x,B34_y,B34_vx,B34_vy,B34_speed,B34_accel,B35_x,B35_y,B35_vx,B35_vy,B35_speed,B35_accel,ball_x,ball_y,ballInPlay
0,1,1,0.1,1,0,A,NaN,NaN,NaN,91.49976,38.0556,0.0,0.0,0.0,0.0,70.459875,17.713350,-0.071801,-1.471252,1.473003,-0.971570,71.815950,33.517350,-0.534203,-0.151597,0.555297,0.472244,73.543545,42.525000,-0.444226,-0.495043,0.665134,-1.996977,67.372155,58.048560,-0.771250,0.148003,0.785323,-0.262673,54.522855,11.192220,-0.296109,-0.311382,0.429697,-1.138126,65.651175,32.400270,-0.319046,0.418088,0.525916,-0.659212,54.188190,32.740200,-0.084754,1.553790,1.556100,-2.972459,66.955140,43.588710,-0.268255,0.677334,0.728520,-1.573195,55.958580,55.531800,-0.480321,-0.553640,0.732956,-0.977324,54.593595,36.471690,0.219643,-0.595247,0.634478,-1.940730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.7588,35.7876,0.0,0.0,0.0,0.0,36.966780,57.469950,0.078251,0.129162,0.151017,-0.308812,31.124385,49.928490,0.259957,-0.167389,0.309187,1.357368,32.878575,30.814110,-1.087167,-0.365654,1.147012,-1.503122,35.022915,16.113960,-0.768273,-0.786708,1.099615,-2.779477,47.995065,44.823600,-0.127491,-0.575637,0.589587,-1.351388,44.686890,38.015640,-0.080743,1.981077,1.982722,-2.928275,45.081090,34.518420,-0.255039,-0.555075,0.610862,-1.021945,44.456040,26.173350,-0.049030,-0.711380,0.713068,1.173747,50.811975,52.713180,1.222197,0.843206,1.484844,-0.766918,52.750170,26.180190,-0.536658,-0.602870,0.807127,-1.771434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.12339,35.08146,0
1,2,1,0.2,1,0,A,NaN,NaN,NaN,91.49976,38.0556,0.0,0.0,0.0,0.0,70.381224,17.500752,-0.106912,-1.335743,1.340015,-1.387549,71.776476,33.477840,-0.165739,-0.358251,0.394732,0.640638,73.509336,42.469560,-0.282102,-0.392744,0.483559,-1.442673,67.304736,58.038984,-0.744403,0.155629,0.760497,-0.298681,54.429732,11.030472,-0.247003,-0.242109,0.345872,-0.744949,65.623932,32.317488,-0.196424,0.396758,0.442718,-0.547116,54.129168,32.990688,-0.081520,1.263730,1.266356,-2.697232,66.932460,43.495560,-0.250726,0.518613,0.576041,-1.243867,55.914840,55.467936,-0.361413,-0.534276,0.645035,-0.693743,54.608688,36.378792,0.155680,-0.512655,0.535772,-0.473780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<ipython-input-96-4c194dd051f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_event_game_3.replace({'team': {'Away': 'Home', 'Home': 'Away'}}, inplace=True)


,team,type,subtype,session,start_frame,start_time,end_frame,end_time,from,to,start_x,start_y,end_x,end_y,phase,goal,ownGoal,teamId,eventName,subEventName,accurate,freeze_frame,value_label,game_id,pass_id
3680,Home,SET PIECE,KICK OFF,1,145,14.5,145,14.5,A10,NaN,NaN,NaN,NaN,NaN,1,0,0,2,Set piece,Kick off,0,"{'A11': {'teammate': True, 'actor': False, 'ba...",0,3,3680
3681,Home,PASS,PASS,1,145,14.5,151,15.1,A10,A07,0.50125,0.48725,0.49864,0.48705,1,0,0,2,Pass,Pass,1,"{'A11': {'teammate': True, 'actor': False, 'ba...",0,3,3681
3682,Home,CARRY,CARRY,1,151,15.1,154,15.4,A07,NaN,0.49864,0.48705,0.49700,0.48500,1,0,0,2,Carry,Carry,0,"{'A11': {'teammate': True, 'actor': False, 'ba...",0,3,3682
3683,Home,PASS,PASS,1,154,15.4,171,17.1,A07,A08,0.49700,0.48500,0.63373,0.63449,1,0,0,2,Pass,Pass,1,"{'A11': {'teammate': True, 'actor': False, 'ba...",0,3,3683
3684,Home,CARRY,CARRY,1,171,17.1,186,18.6,A08,NaN,0.63373,0.63449,0.66986,0.59707,1,0,0,2,Carry,Carry,0,"{'A11': {'teammate': True, 'actor': False, 'ba...",0,3,3684


In [97]:
# get all passes
df_pass = df_event_game_3[
                (df_event_game_3["eventName"] == "Pass")
                & (df_event_game_3["start_frame"] < df_event_game_3["end_frame"])
                & (df_event_game_3["start_frame"] >= 2)
            ].copy()


# only keep relevant columns
df_pass = df_pass[["team", "start_frame", "end_frame", "eventName", "subEventName", "from", "to", "freeze_frame" ,"accurate", "pass_id"]].copy()

# attach the ball position at the start and end frame
df_ball = df_match_3[["frame", "ball_x", "ball_y", "episode", "ballInPlay",]].copy()

df_ball.columns = ["start_frame", "xPosStart", "yPosStart", "episodeStart", "ballInPlayStart"]
df_pass = pd.merge(df_pass, df_ball, how="left")

df_ball.columns = ["end_frame", "xPosEnd", "yPosEnd", "episodeEnd", "ballInPlayEnd"]
df_pass = pd.merge(df_pass, df_ball, how="left")
df_pass = df_pass[df_pass["xPosEnd"].notnull()].copy()

# only keep passes for which the ball was in play at the beginning of the pass (i.e. exclude throw-ins)
df_pass = df_pass[df_pass["ballInPlayStart"] == 1].copy()
df_pass.reset_index(inplace=True, drop=True)

# only keep accurate passes
df_pass_acc = df_pass[df_pass["accurate"] == 1].copy()

display(df_pass_acc.head(20))
print(f'정확한 패스의 개수 : {len(df_pass_acc)}/{len(df_pass)}')

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd
0,Home,145,151,Pass,Pass,A10,A07,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3681,54.123390,35.081460,1,1,53.873208,35.068680,1,1
1,Home,154,171,Pass,Pass,A07,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3683,53.761860,34.987320,1,1,68.254380,45.540792,1,1
2,Home,186,203,Pass,Pass,A08,A02,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3685,72.095724,43.207416,1,1,86.694624,29.017512,1,1
3,Home,212,232,Pass,Pass,A02,A03,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3687,87.316704,30.827592,1,1,86.325804,58.014720,1,1
4,Home,240,252,Pass,Pass,A03,A04,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3689,85.836240,59.311152,1,1,73.595952,70.548696,1,1
5,Home,261,277,Pass,Pass,A04,A03,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3691,73.291392,70.332264,1,1,87.562944,62.475624,1,1
6,Home,289,302,Pass,Pass,A03,A11,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3693,87.878196,62.049168,1,1,103.105116,45.527544,1,1
7,Home,332,345,Pass,Pass,A11,A02,"{'A11': {'teammate': True, 'actor': True, 'bal...",1,3695,102.350412,36.187848,1,1,100.249056,14.693040,1,1
8,Away,450,460,Pass,Pass,B21,B25,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3699,64.083725,-0.523386,1,1,70.803828,6.670584,1,1
9,Away,460,472,Pass,Pass,B25,B21,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3700,70.803828,6.670584,1,1,64.029960,1.902024,1,1


정확한 패스의 개수 : 1121/1243


In [98]:
# Intended Receiver 예측

intended_list = []
frame_dist = []
frame_receiver = []
frame_ball = []
frame_angle = []

# match dataframe 에서 선수들 위치 정보 추출 함수
def extract_player_pos(df, frame):

  basic = df[df['frame'] == frame]

  # player 전처리
  player_df = []
  for i in range(1, 29):
    if i < 10:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'0{i}').iloc[0])[0:2]
      i_basic += ['Home']
    else:
      i_basic = [i]
      i_basic += list(basic.filter(regex=f'{i}').iloc[0])[0:2]
      if i <= 14 :
        i_basic += ['Home']
      else:
        i_basic += ['Away']
    player_df.append(i_basic)
  player_df = pd.DataFrame(player_df, columns=['playerId', 'xPos', 'yPos', 'team']).dropna(axis=0)

  return player_df


for idx, action in tqdm(df_pass_acc.iterrows()):
  # get coordinates of the pass start location, interception point
  # and each potential receiver
  df_start_frame = extract_player_pos(df_match_3, action.start_frame)
  receiver_coo = np.array(
      [
          (o["xPos"], o["yPos"])
          for index, o in df_start_frame.iterrows()
      ]
  )

  len_home = len(df_start_frame[df_start_frame['team'] == 'Home'])
  if action['team'] == 'Home':
    receiver_coo = receiver_coo[:len_home]
  else:
    receiver_coo = receiver_coo[len_home:]

  ball_coo = np.array([action.xPosStart, action.yPosStart])
  interception_coo = np.array([action.xPosEnd, action.yPosEnd])

  # compute the distance between the location where the ball was
  # intercepted and each potential receiver
  dist = np.sqrt(
                (receiver_coo[:, 0] - interception_coo[0]) ** 2
                + (receiver_coo[:, 1] - interception_coo[1]) ** 2
            )

  frame_dist.append(dist)
  frame_ball.append(ball_coo)
  frame_receiver.append(receiver_coo)

  # compute the angle between each potential receiver and the passing line
  a = interception_coo - ball_coo
  b = receiver_coo - ball_coo
  angle = np.arccos(
                np.clip(
                    np.sum(a * b, axis=1) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1)), -1, 1
                )
            )


  # if not players are in 20 degrees of the pass line, the intended
  # receiver was probably not in the freeeze frame
  if np.amin(angle) > 0.35:
    intended_list.append(0)
    continue
  # only consider players in 20 degrees of the pass line
  too_wide = np.where(angle > 0.35)[0]
  dist[too_wide] = np.inf

  # find the most likely intended receiver
  # TODO: you could play around with the weight given to the distance
  # and angle here
  exp_receiver = np.argmax((np.amin(dist) / dist) * (np.amin(angle) / angle))

  # end_frame 에서 선수들 모두 리스트업
  id_list = list(df_start_frame["playerId"])

  # Home, Away 구분
  if action['from'][0] == "A" :
    id = id_list[:len_home]
  else:
    id = id_list[len_home:]

  try:
    intended_list.append(id[exp_receiver])
  except:
    print(f'idx : {idx}')
    print(f'len_home : {len_home}')
    print(f'receiver 수 : {len(receiver_coo)}')
    print(f'총 선수 수 : {len(id_list)}')
    print(f'타겟 팀 선수 수 : {len(id)}')
    print(f'오차 : {len(receiver_coo) - len(id)}')

df_pass_acc['intended_receiver'] = intended_list

1121it [00:22, 49.13it/s]


In [99]:
# 기존 데이터와의 Accuracy 실험
accurate_count = 0

err_str_frame = []
err_end_frame = []
err_idx = []
for idx, row in df_pass_acc.iterrows():
  toplayerId = int(row['to'][1:])
  if toplayerId == row['intended_receiver']:
    accurate_count += 1
  else:
    err_str_frame.append(row['start_frame'])
    err_end_frame.append(row['end_frame'])
    err_idx.append(idx)

accuracy_rate = accurate_count / len(df_pass_acc) * 100

print(f'Game 3에 대한 intended receiver 정확도 : {accuracy_rate}')

Game 3에 대한 intended receiver 정확도 : 71.54326494201607


In [100]:
match_3_err = df_pass_acc[df_pass_acc['start_frame'].isin(err_str_frame)]
display(match_3_err.head(10))

match_3_err.to_csv("/content/drive/MyDrive/Data Science/Colab Notebooks/Study/EPV_KCC/soccer_analytics-master/data/match_3_err.csv")

,team,start_frame,end_frame,eventName,subEventName,from,to,freeze_frame,accurate,pass_id,xPosStart,yPosStart,episodeStart,ballInPlayStart,xPosEnd,yPosEnd,episodeEnd,ballInPlayEnd,intended_receiver
0,Home,145,151,Pass,Pass,A10,A07,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3681,54.123390,35.081460,1,1,53.873208,35.068680,1,1,0
7,Home,332,345,Pass,Pass,A11,A02,"{'A11': {'teammate': True, 'actor': True, 'bal...",1,3695,102.350412,36.187848,1,1,100.249056,14.693040,1,1,11
10,Home,526,548,Pass,Pass,A06,A07,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3711,68.487012,15.415272,1,1,60.330852,27.504144,1,1,0
20,Home,791,800,Pass,Pass,A10,A05,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3733,54.670896,4.368960,1,1,58.168044,5.449392,1,1,2
21,Away,829,849,Pass,Pass,B20,B23,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3738,40.036032,14.900400,1,1,40.549896,26.741376,1,1,18
22,Away,873,899,Pass,Pass,B23,B19,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3741,40.787064,30.010320,1,1,22.984668,42.632352,1,1,0
29,Away,1216,1240,Pass,Pass,B25,B21,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3761,75.542976,39.387888,2,1,60.165720,32.824080,2,1,25
30,Away,1240,1254,Pass,Pass,B21,B20,"{'B28': {'teammate': True, 'actor': False, 'ba...",1,3763,60.165720,32.824080,2,1,45.824724,36.635760,2,1,28
59,Home,3384,3404,Pass,Pass,A02,A06,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3865,81.884520,36.907560,6,1,70.468272,44.673336,6,1,8
65,Home,3597,3608,Pass,Pass,A05,A08,"{'A11': {'teammate': True, 'actor': False, 'ba...",1,3881,62.895852,16.493256,6,1,62.526600,25.880328,6,1,4


**틀린 예측 - 영상**

In [ ]:
# look at the pass starting in frame 498
frame = 498
viz = viz_help.trackingDataVisualizer(df_track, df_events)
viz.set_sequence_by_frames(start_frame=frame, end_frame=frame + 40)
viz.add_player_number()
fig = viz.get_figure()
plotly.offline.iplot(fig, validate=False, auto_play=False)

NameError: name 'df_track' is not defined

**2) Unsuccess에서 Intended Receiver 성능 비교**

**3) 결과 보고**